In [3]:
%load_ext autoreload
%autoreload 2

from warnings import filterwarnings

import torch
import torch.nn as nn

import config_iProL_test as config
from dataset import RegulonDataset
from dataset import get_excel, get_complete_data, get_split_dataset
from dataset import get_datasets, get_cv_dataloader_list, feature_exaction
from feature_extraction import Kmer
from module import Longformer_base_Net, Longformer_base_lstm_Net, Bert_base_lstm_Net
from tools import get_scoring_result, drawLoss, check_result_dir, get_config_info, save_score
from tools_fit import model_fit



model = Longformer_base_lstm_Net(config.model_name, config.device,
                                 config.add_special_tokens,
                                 bidirectional=config.bidirectional,
                                 num_layers=config.num_layers)
model.load_state_dict(torch.load("../model/model_137.pkl"))

filterwarnings("ignore")

model.to(config.device)

criterion = nn.BCELoss()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some weights of the model checkpoint at /home/pbc/Pycharm Projects/myNewPro/pre-model/longformer/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
sheet = get_excel(version='', sheet_name='dataset')
complete_data = get_complete_data(sheet)
dataset_train, _, _ = get_datasets(complete_data)

sheet = get_excel(version='11.1', sheet_name='test_0.85')
complete_data = get_complete_data(sheet)
dataset_test, _, _ = get_datasets(complete_data)
train_loader, test_loader = feature_exaction(config, [dataset_train, dataset_test], "Pro")

PBC[test] --- kmer = 2
train len: 6764 test len: 395


In [5]:
model.eval()
total_loss_test = 0
y_test = []
y_prob = []
with torch.no_grad():
    for idx_test, (X, y) in enumerate(test_loader, 1):
        try:
            X = X.to(config.device)
        except Exception as e:
            # print(e)
            pass
        y = y.float().to(config.device)
        # print(X.shape, y.shape)
        y_hat = model(X)
        loss = criterion(y_hat, y)
        total_loss_test += loss.item()

        y_test.extend(y.to('cpu').tolist())
        y_prob.extend(y_hat.to('cpu').tolist())
        # if idx_test % 20 == 0:
        #     print(f'epoch {epoch} [{idx_test}/{len(train_loader)}] batch_loss = {loss.item()}')
    y_pred = [1 if item >= 0.5 else 0 for item in y_prob]


    scoring = ['accuracy', 'matthews_corrcoef', 'f1']
    process_msg, test_score_dict = get_scoring_result(scoring, y_test, y_pred, y_prob)
    print(process_msg)
    test_loss_epoch = total_loss_test / idx_test * 1.0
    # return test_loss_epoch, test_score_dict

total=395, TP=367, TN=0, FP=0, FN=28; precision=1.000, recall=0.929, Sn=0.929, Sp=nan
accuracy: (test=0.929) matthews_corrcoef: (test=0.000) f1: (test=0.963) 


In [10]:
import os
file = "./iProL_re.txt"
with open(file,'w') as f:
    # if not os.path.exists(file):
    #     os.path.
    f.writelines([str(p)+"\n" for p in y_pred])
    f.close()